sửa bleu thành corpus_bleu

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Nov 26 04:14:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp -av '/content/drive/MyDrive/coco/pycocotools' '/content/'

# change the current working directory to the parent directory of this code file # hadi
import os
os.chdir('/content/drive/MyDrive/YOLOv4_CNN_RNN')  # hadie

'/content/drive/MyDrive/coco/pycocotools' -> '/content/pycocotools'
'/content/drive/MyDrive/coco/pycocotools/_mask.pyx' -> '/content/pycocotools/_mask.pyx'
'/content/drive/MyDrive/coco/pycocotools/__init__.py' -> '/content/pycocotools/__init__.py'
'/content/drive/MyDrive/coco/pycocotools/_mask.c' -> '/content/pycocotools/_mask.c'
'/content/drive/MyDrive/coco/pycocotools/_mask.cp39-win_amd64.pyd' -> '/content/pycocotools/_mask.cp39-win_amd64.pyd'
'/content/drive/MyDrive/coco/pycocotools/mask.py' -> '/content/pycocotools/mask.py'
'/content/drive/MyDrive/coco/pycocotools/coco.py' -> '/content/pycocotools/coco.py'
'/content/drive/MyDrive/coco/pycocotools/__pycache__' -> '/content/pycocotools/__pycache__'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/coco.cpython-39.pyc' -> '/content/pycocotools/__pycache__/coco.cpython-39.pyc'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/__init__.cpython-39.pyc' -> '/content/pycocotools/__pycache__/__init__.cpython-39.pyc'
'/content/drive/My

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # hadie
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on  during captioning

import matplotlib.pyplot as plt
from pycocotools.coco import COCO

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

from tqdm import tqdm  # hadie
from termcolor import colored  # hadie
from builtins import len  # hadie
import datetime  # hadie

from model import DATASET, EXAMPLE_NUMBER, LIMIT_SIZE , WORD_DICT_SIZE, TEST_SET_PROPORTION, MY_EMBEDDING_DIM, UNIT_COUNT, CNN_Encoder, RNN_Decoder, MY_OPTIMIZER, MY_LOSS_OBJECT, REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN, EPOCH_COUNT, feature_extraction_model, split  # hadie
from timeit import default_timer as timer  # hadie
import threading  # hadie

# from yolo import image_path_to_yolo_bounding_boxes  # hadie
import cv2


In [ ]:
# Hong
# 1. Load COCO training data

PATH ='/content/drive/MyDrive/coco/train2014/'
annotation_file = "/content/drive/MyDrive/coco/annotations/captions_train2014.json"

coco = COCO(annotation_file)
all_Ids = coco.getImgIds()
input_ids = shuffle(all_Ids, random_state=0)

num_examples = 20000
input_ids = input_ids[:num_examples]

input_img_name = []
for i in range(len(input_ids)):
    input_img_name.append(PATH+'COCO_train2014_' + '%012d' % (input_ids[i]) + '.jpg')

### input_data = {img_name_train[i]: [cap_1, cap_2, ... cap_5]}
input_data = {}
for i in range(0,len(input_img_name)):
    if input_img_name[i] not in input_data:
        annIds = coco.getAnnIds(imgIds = input_ids[i])
        anns = coco.loadAnns(annIds)
        captions = []
        for j in range(0,len(anns)):
            a = '<start> '+ anns[j]['caption'] + ' <end>'
            captions.append(a.lower())
        input_data[input_img_name[i]] = captions

print(input_img_name[:1])
print(input_ids[:1])

loading annotations into memory...
Done (t=2.92s)
creating index...
index created!
['/content/drive/MyDrive/coco/train2014/COCO_train2014_000000197705.jpg']
[197705]


In [ ]:
"""## Preprocess the images
"""

# the load_image function has been moved to a separate file by hadie

import importlib  # hadie
mod = importlib.import_module("feature_extraction_model_" + feature_extraction_model)  # hadie
image_model = mod.image_model  # hadie
load_image = mod.load_image  # hadie
attention_features_shape = mod.attention_features_shape + 1  # hadie

83683744/83683744 [==============================] - 5s 0us/step


In [ ]:
"""
"""

# the image_model variable has been moved to a separate file by hadie

new_input = image_model.input # (none, none, none, 3)
hidden_layer = image_model.layers[-1].output # (None, None, None, 2048)

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
print('Params for Xception ',image_features_extract_model.count_params())

Params for Xception  20861480


In [ ]:
from scipy.linalg.decomp import argsort
from IPython.utils.text import indent

# Yolo model
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')
scale = 0.00392 #=1/255
layer_names = ['yolo_139','yolo_150','yolo_161'] # 3 head layers

def extract_yolo_features(filename):

    # net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')
    # scale = 0.00392 #=1/255

    image = cv2.imread(filename)

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)

    # layer_names = ['yolo_139','yolo_150','yolo_161'] # 3 head layers
    layers = net.forward(layer_names)

    return layers


def extract_combined_feature(filename):

    # 1. Xception feature
    img, _ = load_image(filename)
    img = np.expand_dims(img, axis=0)
    # print('img.shape',img.shape) # (1, 299, 299, 3)
    batch_features = image_features_extract_model(img)
    # print('batch_features 1', batch_features.shape) # (1, 10, 10, 2048)
    batch_features = tf.reshape(batch_features,
                                (-1, batch_features.shape[3]))
    # print('batch_features 2', batch_features.shape) # (100, 2048)

    # 2. Yolo objects
    extracted_layers = extract_yolo_features(filename)

    # concat 3 output layers
    out = np.concatenate((extracted_layers[0], extracted_layers[1], extracted_layers[2]), axis = 0)

    ### NMS to get bounding boxes

    image = cv2.imread(filename)
    # Width = image.shape[1]
    # Height = image.shape[0]

    class_ids=[]
    confidences=[]
    p = []
    boxes=[]
    conf_threshold, nms_threshold = 0.5, 0.4

    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id] # prop of class
        if confidence > conf_threshold:
            # center_x = int(detection[0] * Width)
            # center_y = int(detection[1] * Height)
            # w = int(detection[2] * Width)
            # h = int(detection[3] * Height)
            # x = center_x - w / 2
            # y = center_y - h / 2

            p.append(detection[4])

            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([detection[0], detection[1], detection[2], detection[3]])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    # print('indices: ', indices)

    bounding_boxes=[]
    important=[]
    for i in range(len(list(indices))):
        # boxes = [x, y, w, h, p, class_ids, important_factor]

        ########## can takeover this 4 position values to save more [class_id, important_factor]
        bbx = boxes[indices[i]].copy()

        bbx.append(p[indices[i]])

        important_factor = boxes[indices[i]][2] * boxes[indices[i]][3] * confidences[indices[i]]
        important.append(important_factor)
        bbx.append(class_ids[indices[i]])
        bbx.append(important_factor)

        bounding_boxes.append(bbx)

    ### sort bounding box to get the most important bounding box first

    ranked = np.argsort(np.array(important))
    largest_indices = ranked[::-1]

    sorted_bbx=[]
    for i in largest_indices:
        b = bounding_boxes[i].copy()
        sorted_bbx.append(b)
    # print('sorted_bbx',sorted_bbx)

    # 3. Combine feature and objects

    # flattening and padding to fit size of feature vector

    features_shape = batch_features.shape[1] # 2048
    # print('features_shape',features_shape) # 2048

    yolo_features = np.array(sorted_bbx).flatten()
    # print('shape of yolo_features', yolo_features.shape)

    # get the first num_of_box to fit the size of feature_shape
    num_of_box = features_shape//7          #

    yolo_features = yolo_features[:num_of_box*7]
    # print('yolo_features.shape ', yolo_features.shape)
    yolo_features = np.pad(yolo_features, (0, features_shape - yolo_features.shape[0]), 'constant', constant_values=(0, 0)).astype(np.float32)
    yolo_features = yolo_features.reshape(-1, yolo_features.shape[0])
    # print('yolo_features.shape',yolo_features.shape)

    feature = np.concatenate((batch_features, yolo_features), axis=0) # 101x2048
    # print('combined feature shape', feature.shape)

    return feature

<ipython-input-8-130f2b41b8a3>:1: DeprecationWarning: Please use `argsort` from the `scipy.linalg` namespace, the `scipy.linalg.decomp` namespace is deprecated.
  from scipy.linalg.decomp import argsort


In [ ]:
# extract combined feature for each image

print("-----------------------------START OF EXECUTION-----------------------------")  # hadie

feature_dir = './3_Yolo4_Xception_RNN/xception_npy'

encode_train = [x for x in input_img_name if not os.path.exists(feature_dir + "/" + x[-31:-4] + ".npy")]
# features_shape = 2048
print("extracting features (" + str(len(encode_train)) + ") file(s)")
print('len of extracting',len (encode_train))

if len(encode_train) > 0:
    image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
    image_dataset = image_dataset.batch(16)

    for paths in tqdm(image_dataset):
        for path in paths:
            # print('path', path)
            fn = path.numpy().decode("utf-8")
            combined_feature = extract_combined_feature(fn)
            np.save(feature_dir + "/" + fn[-31:-4], combined_feature)

print("\nfinished extracting features")

-----------------------------START OF EXECUTION-----------------------------
extracting features (0) file(s)
len of extracting 0

finished extracting features


In [ ]:
"""## Preprocess and tokenize the captions

* First, you'll tokenize the captions (for example, by splitting on spaces).
This gives us a  vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).
* Next, you'll limit the vocabulary size to the top 5,000 words (to save memory).
You'll replace all other words with the token "UNK" (unknown).
* You then create word-to-index and index-to-word mappings.
* Finally, you pad all sequences to be the same length as the longest one.
"""
# Hong
def dict_to_list(train_descriptions):
    all_desc = []
    for key in train_descriptions.keys():
        [all_desc.append(d) for d in train_descriptions[key]]
    return all_desc

# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


# Choose the top 5000 words from the vocabulary
top_k = WORD_DICT_SIZE  # hadie 15000



##################################################
REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN = False

input_captions = dict_to_list(input_data) # Hong

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    print("using the cashed tokenizer")  # hadie
    # loading the tokenizer # hadie
    with open("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'rb') as handle:  # hadie
        tokenizer = pickle.load(handle)  # hadie
else:  # hadie
    print("tokenizing and padding captions")  # hadie
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                      oov_token="<unk>",
                                                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(input_captions)
    # train_seqs = tokenizer.texts_to_sequences(train_captions)  # 777 maybe this line needs removal
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    # saving the tokenizer to disk # hadie
    with open("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'wb') as handle:  # hadie
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  # hadie



using the cashed tokenizer


In [ ]:
# Create training and validation sets using an 80-20 split
PROP = 0.1 # hong

if PROP == 0:
    image_id_train = input_ids
    image_id_val = []
    img_name_train = input_img_name
    img_name_val = []
    # cap_train = cap_vector
    # cap_val = []
elif PROP == 1:
    image_id_train = []
    image_id_val = input_ids
    img_name_train = []
    img_name_val = input_img_name
    # cap_train = []
    # cap_val = cap_vector
else:
    image_id_train, image_id_val, img_name_train_2, img_name_val_2 = train_test_split(
                                                                    input_ids,  # hadie
                                                                    input_img_name,
                                                                    test_size=PROP,  # hadie
                                                                    random_state=0)

cap_train_list = []
cap_val_list = []
for i in range(len(img_name_train_2)):
    for j in range(len(input_data[img_name_train_2[i]])):
        cap_train_list.append(input_data[img_name_train_2[i]][j])
for i in range(len(img_name_val_2)):
    for j in range(len(input_data[img_name_val_2[i]])):
        cap_val_list.append(input_data[img_name_val_2[i]][j])

train_seqs = tokenizer.texts_to_sequences(cap_train_list)
val_seqs = tokenizer.texts_to_sequences(cap_val_list)
cap_train = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
cap_val = tf.keras.preprocessing.sequence.pad_sequences(val_seqs, padding='post')

# Hong
def dict_to_list_2(dict_data, list_key):
    all_img_name = []
    for i in range(len(list_key)):
        if list_key[i] in dict_data:
            for j in range(len(dict_data[list_key[i]])):
                all_img_name.append(list_key[i])
    return all_img_name

img_name_train = dict_to_list_2(input_data, img_name_train_2)
img_name_val = dict_to_list_2(input_data, img_name_val_2)



In [ ]:
if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    file = "./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'r') as filetoread:  # hadie
        max_length = int(filetoread.readline())  # hadie
else:  # hadie
    # Calculates the max_length, which is used to store the attention weights
    max_length = calc_max_length(train_seqs)

    file = "./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'w') as filetowrite:  # hadie
        filetowrite.write(str(max_length))  # write the maximum length to disk # hadie

print("finished tokenizing and padding captions")  # hadie

finished tokenizing and padding captions


In [ ]:
"""## Create a tf.data dataset for training
"""

# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 8*5  # 64
# BUFFER_SIZE = 10000
embedding_dim = MY_EMBEDDING_DIM  # hadie 256
units = UNIT_COUNT  # hadie 512
vocab_size = top_k + 1 # 15001
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
# the definition of features_shape = 2048 was moved up
# the attention_features variable has been moved to a separate file by hadie


# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(feature_dir + '/' + img_name.decode('utf-8')[-31:-4] + '.npy')
  return img_tensor, cap


dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
dataset = dataset.shuffle(BATCH_SIZE * 8)

dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
"""## Model

"""

# the model has been moved to model.py by hadie

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = MY_OPTIMIZER  # hadie
loss_object = MY_LOSS_OBJECT  # hadie


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


"""## Checkpoint"""

checkpoint_path = "./3_Yolo4_Xception_RNN/checkpoint"

if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:
    try:
        for filename in os.listdir(checkpoint_path):
            print("deleting " + checkpoint_path + "/" + filename)
            os.unlink(checkpoint_path + "/" + filename)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (checkpoint_path + "/" + filename, e))
    # remove the saved model too
    if os.path.exists("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model.index"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.index")
        os.unlink("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model.index")
    if os.path.exists("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/checkpoint"):
        print("deleting /trained_model_" + feature_extraction_model + "/checkpoint")
        os.unlink("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/checkpoint")
    if os.path.exists("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
        os.unlink("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
    if os.path.exists("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/learning_curve.png"):
        print("deleting trained_model_" + feature_extraction_model + "/learning_curve.png")
        os.unlink("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/learning_curve.png")

ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    ckpt.restore(ckpt_manager.latest_checkpoint)


In [ ]:
"""## Training

"""

# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []


@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)
    #   print('Count params of encoder: ',encoder.count_params())

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)
        #   print('Count params of decoder', decoder.count_params())

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss


# EPOCHS = EPOCH_COUNT  # hadie
EPOCHS = 10

if not os.path.exists("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model.index"):  # hadie
    print("training..")  # hadie

    t1 = time.time()
    for epoch in range(start_epoch, EPOCHS):
        start = time.time()
        total_loss = 0

        for (batch, (img_tensor, target)) in enumerate(dataset):
            # print('img_tensor.shape', img_tensor.shape)
            # print('target.shape', target.shape)
            batch_loss, t_loss = train_step(img_tensor, target)
            total_loss += t_loss

            if batch % 100 == 0:
                print ('Epoch {} Batch {} Loss {:.4f}'.format(
                  epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
        # storing the epoch end loss value to plot later
        loss_plot.append(total_loss / num_steps)

        # if epoch % 5 == 0:
        #   ckpt_manager.save()
        ckpt_manager.save()

        print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                             total_loss / num_steps))
        print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    t2 = time.time()
    # save model to disk # hadie
    decoder.save_weights("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model" , save_format="tf")  # hadie
    print('Time of training: ', t2-t1)
else:  # hadie
    print("A trained model has been found. Loading it from disk..")  # hadie
    # Load the previously saved weights # hadie
    decoder.load_weights("./3_Yolo4_Xception_RNN/trained_model_" + feature_extraction_model + "/my_model")  # hadie

# showing the learning curve is moved from here by hadie

training..
Epoch 3 Batch 0 Loss 0.6997
Epoch 3 Batch 100 Loss 0.6033
Epoch 3 Batch 200 Loss 0.6791
Epoch 3 Batch 300 Loss 0.6650
Epoch 3 Batch 400 Loss 0.7420
Epoch 3 Batch 500 Loss 0.6520
Epoch 3 Batch 600 Loss 0.6604
Epoch 3 Batch 700 Loss 0.5789
Epoch 3 Batch 800 Loss 0.5833
Epoch 3 Batch 900 Loss 0.5264
Epoch 3 Batch 1000 Loss 0.5876
Epoch 3 Batch 1100 Loss 0.5303
Epoch 3 Batch 1200 Loss 0.5545
Epoch 3 Batch 1300 Loss 0.5847
Epoch 3 Batch 1400 Loss 0.5767
Epoch 3 Batch 1500 Loss 0.5941
Epoch 3 Batch 1600 Loss 0.6301
Epoch 3 Batch 1700 Loss 0.5621
Epoch 3 Batch 1800 Loss 0.6025
Epoch 3 Batch 1900 Loss 0.6151
Epoch 3 Batch 2000 Loss 0.5652
Epoch 3 Batch 2100 Loss 0.6030
Epoch 3 Batch 2200 Loss 0.5574
Epoch 3 Loss 0.611892
Time taken for 1 epoch 2246.9666912555695 sec

Epoch 4 Batch 0 Loss 0.6412
Epoch 4 Batch 100 Loss 0.6377
Epoch 4 Batch 200 Loss 0.5568
Epoch 4 Batch 300 Loss 0.6252
Epoch 4 Batch 400 Loss 0.6006
Epoch 4 Batch 500 Loss 0.6424
Epoch 4 Batch 600 Loss 0.5225
Epoch 4 Bat

In [ ]:
"""## Caption!

"""

attention_features_shape = 101
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    combined_features = extract_combined_feature(image)
    features = encoder(combined_features)  # hadie

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        # print('predictions', predictions)

        attention_plot[i] = tf.reshape(attention_weights, (-1,)).numpy()
        # print('attention_plot[i]', attention_plot[i])

        # predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        predicted_id = np.argmax(predictions.numpy())
        # print('predicted_id', predicted_id)

        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result),:]
    return result, attention_plot


def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

def list_to_dict(name_list, caption_list):
    name_dict = {}
    for i in range(len(name_list)):
        if name_list[i] not in name_dict:
            name_dict[name_list[i]] = [caption_list[i]]
        else:
            name_dict.get(name_list[i]).append(caption_list[i])
            # print( name_dict.get(name_list[i]))
    return name_dict

# captions on the validation set
val_dict = list_to_dict(img_name_val, cap_val)


# # for random samples
# for i in range(10):
#     # rid = np.random.randint(0, len(img_name_val))
#     image = img_name_val[i*5]

#     captions = val_dict[image]
#     # print('captions:', captions)

#     result, attention_plot = evaluate(image)

#     # print ('Real Caption:', real_caption)
#     print('Real captions:')
#     for j in range(len(captions)):
#         real_caption = ' '.join([tokenizer.index_word[i] for i in captions[j] if i not in [0]])
#         print(real_caption)

#     print ('Prediction Caption:', ' '.join(result))
#     # plot_attention(image, result, attention_plot)

#     print('img path', image)

#     import cv2
#     from google.colab.patches import cv2_imshow
#     img = cv2.imread(image)
#     cv2_imshow(img)


# for some first samples

list_img = [
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000127298.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000031135.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000448859.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000215718.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000532398.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000567901.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000207691.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000292135.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000304973.jpg',
    '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000484734.jpg'
]

for image in list_img:
    # rid = np.random.randint(0, len(img_name_val))
    # image = img_name_val[i*5]

    captions = val_dict[image]
    # print('captions:', captions)

    result, attention_plot = evaluate(image)

    # print ('Real Caption:', real_caption)
    print('Real captions:')
    for j in range(len(captions)):
        real_caption = ' '.join([tokenizer.index_word[i] for i in captions[j] if i not in [0]])
        print(real_caption)

    print ('Prediction Caption:', ' '.join(result))
    # plot_attention(image, result, attention_plot)

    print('img path', image)

    import cv2
    from google.colab.patches import cv2_imshow
    img = cv2.imread(image)
    cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


def img_to_caption(img_name):
    result, attention_plot = evaluate(img_name)
    result = ' '.join(result).replace("<end>", "").strip()
    return result

actual, predicted = [], []

for img_name in tqdm(val_dict):

    yhat = img_to_caption(img_name)
    pred = yhat.split()

    actu = []
    captions = val_dict[img_name]
    for i in range(len(captions)):
        real_caption = ' '.join([tokenizer.index_word[i] for i in captions[i] if i not in [0,3,4]])
        references = real_caption.split(' ')
        actu.append(references)

    # print('references:', references)
    # print('predicted:', predicted)

    actual.append(actu)
    predicted.append(pred)

bl1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
bl2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
bl3 = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
bl4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

# print('actual:', actual)
# print('predicted: ' ,predicted)

mt = 0
for i in range(len(predicted)):
    mt += round(meteor_score(actual[i], predicted[i]),4)
mt = mt/len(predicted)
# print('METEOR: %f' % mt)

print("\nbleu-1 score:", bl1)
print("bleu-2 score:", bl2)
print("bleu-3 score:", bl3)
print("bleu-4 score:", bl4)
print("meteor score:", mt)


In [ ]:
# speed test

# for random samples
img_list = []
for i in range(100):
    rid = np.random.randint(0, len(img_name_val))
    img_list.append(img_name_val[rid])

t1 = time.time()
for image in img_list:
    result, attention_plot = evaluate(image)
    predict = ' '.join(result)

t2 = time.time()
print('time of predicting captions for 100 images', t2-t1)

In [ ]:
# # speed test => moved to local test

# file = open('./save/img_name_val_bk.txt', 'r')
# txt = file.read()
# file.close()

# test_imgs1 = txt[1:-1].split(", ")
# test_imgs = []
# num_test_file = 5
# for i in range(num_test_file):
#     test_imgs.append(test_imgs1[i][1:-1])

# # ['/content/drive/MyDrive/coco/train2014/COCO_train2014_000000151458.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000419159.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000239584.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000521266.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000337415.jpg']

# time_extract = 0
# time_predict = 0

# def predict_caption(image):

#     time_start_extract = time.time()
#     combined_features = extract_combined_feature(image)
#     t = time.time() - time_start_extract

#     features = encoder(combined_features)

#     hidden = decoder.reset_state(batch_size=1)
#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

#     result = []
#     for i in range(max_length):
#         predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

#         predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

#         # hong
#         while predicted_id > len(tokenizer.index_word)-1:
#             predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
#         # hong end

#         result.append(tokenizer.index_word[predicted_id])

#         if tokenizer.index_word[predicted_id] == '<end>':
#             # print('early')
#             return result, t

#         dec_input = tf.expand_dims([predicted_id], 0)

#     return result, t


# # captions on the validation set
# n = len(test_imgs)
# for i in range(n):
#     image = test_imgs[i]

#     time_start_predict = time.time()
#     result, t = predict_caption(image)
#     time_predict += time.time() - time_start_predict
#     time_extract += t

#     print ('Prediction Caption:', ' '.join(result))

#     # print('img path', image)
#     # import cv2
#     # from google.colab.patches import cv2_imshow
#     # img = cv2.imread(image)
#     # cv2_imshow(img)

# time_extract /= n
# time_predict /= n
# print('average time_extract by Yolo_xception_RNN', time_extract)
# print('average time_predict by Yolo_xception_RNN', time_predict)